# Number of photons in laser pulse, as a function of the number of laser slices

_________

#### Imports

In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal

import scipy.constants as const

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

### Load File Data

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')
meta_file_name = 'wfs_meta.dat'
ccd_name = 'ccd_pump_off.txt'
wfs_name = 'wfs_pump_off.txt'
meta_path_to_file = os.path.join(package_data_dir, meta_file_name)
ccd_path_to_file = os.path.join(package_data_dir, ccd_name)
wfs_path_to_file = os.path.join(package_data_dir, wfs_name)

files = PKDict(
    meta = meta_path_to_file,
    ccd = ccd_path_to_file,
    wfs = wfs_path_to_file
    )

## From Function

In [ ]:
crystal_params = PKDict(length = 2.5/100.0,  # [m]
                        nslice = 5,
)
prop_type = 'gain_calc'

max_num_l_slices = 20

total_n_photons_i = np.zeros(max_num_l_slices)
total_n_photons   = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):

    params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = num_l_slices+1,
        nx_slice = 32,
        ny_slice = 32,
        pulseE = 1.0e-6,
        tau_fwhm = 300.0e-12 / math.sqrt(2.), #0.1 / const.c / math.sqrt(2.),
        sigx_waist = 1.2e-3 /1.18, #1.0e-3,
        sigy_waist = 1.2e-3 /1.18, #1.0e-3,
    )
    
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params)
    nslices_pulse = len(thisPulse.slice)
    nslices_crystal = len(e_crystal.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_l_slices < max_num_l_slices):
        #print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

In [ ]:
if thisPulse.slice[0].ds == thisPulse.slice[-1].ds:
    fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)
else:
    print('Slices are different lengths!')

nslices_pulse = len(thisPulse.slice)
photons_in_slice = np.zeros((nslices_pulse))
for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(fig_x,photons_in_slice)
box = ax.get_position()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

x_function = fig_x
photons_function = photons_in_slice

## From File

In [ ]:
crystal_params = PKDict(length = 2.5/100.0,  # [m]
                        nslice = 5,
)
prop_type = 'gain_calc'

max_num_l_slices = 20

total_n_photons_i = np.zeros(max_num_l_slices)
total_n_photons   = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):

    params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = num_l_slices+1,
        nx_slice = 32,
        ny_slice = 32,
        pulseE = 1.0e-6,
        tau_fwhm = 300.0e-12 / math.sqrt(2.), #0.1 / const.c / math.sqrt(2.),
        sigx_waist = 1.2e-3 /1.18, #1.0e-3,
        sigy_waist = 1.2e-3 /1.18, #1.0e-3,
    )
    
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params, files)
    nslices_pulse = len(thisPulse.slice)
    nslices_crystal = len(e_crystal.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_l_slices < max_num_l_slices):
        #print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

In [ ]:
if thisPulse.slice[0].ds == thisPulse.slice[-1].ds:
    fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)
else:
    print('Slices are different lengths!')

nslices_pulse = len(thisPulse.slice)
photons_in_slice = np.zeros((nslices_pulse))
for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(fig_x,photons_in_slice)
box = ax.get_position()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

x_file = fig_x
photons_file = photons_in_slice

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(x_file,photons_file, 'k', label='File Init')
ax2.plot(x_function,photons_function, '--r', label='Function Init')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Distance from Laser Front [m]')
ax1.set_ylabel(r'Number of Photons, File Init')
ax2.set_ylabel(r'Number of Photons, Function Init',color='r')
plt.title('Longitudinal Photon Distribution')